 # Notebook 06: Test Approfondi du Pipeline de Bout en Bout



 Ce notebook est dédié à un test complet et une analyse détaillée du workflow "MAKERS" sur une requête utilisateur complexe. Nous allons observer les sorties intermédiaires des agents, le flux de décision, et la qualité de la synthèse finale. Nous explorerons également comment inspecter les états sauvegardés par le checkpointer MongoDB.



 **Prérequis :**

 * **Environnement de Base :** Avoir exécuté le notebook `00_setup_environment.ipynb` pour configurer l'environnement Conda, les dépendances Python, et s'assurer que le fichier `.env` à la racine du projet est correctement rempli avec toutes les configurations nécessaires.

 * **Base de Données MongoDB :**

     * `MONGODB_URI` doit être correctement configuré dans `.env` et votre instance MongoDB doit être accessible. Ceci est fondamental pour le checkpointer LangGraph (`MongoDBSaver`) et les outils RAG.

     * La base de données doit être peuplée (via `01_data_ingestion_and_embedding.ipynb` ou `scripts/run_ingestion.py`) avec des documents dont les embeddings ont été générés en utilisant le fournisseur spécifié par `DEFAULT_EMBEDDING_PROVIDER` dans vos paramètres. Assurez-vous que les embeddings sont cohérents avec le fournisseur que vous comptez utiliser pour les requêtes dans ce notebook.

 * **Configuration des Fournisseurs de Modèles (dans `.env`) :** Le workflow `MAKERS` (exécuté par `run_makers_v2_1` dans ce notebook) utilisera les fournisseurs configurés via `DEFAULT_LLM_MODEL_PROVIDER` (pour les agents) et `DEFAULT_EMBEDDING_PROVIDER` (pour la RAG). Vérifiez que les configurations correspondantes sont correctement en place :

     * **Pour les LLMs des Agents (`DEFAULT_LLM_MODEL_PROVIDER`) :**

         * Si réglé sur `"openai"` : `OPENAI_API_KEY` et `DEFAULT_OPENAI_GENERATIVE_MODEL` sont requis.

         * Si réglé sur `"huggingface_api"` : `HUGGINGFACE_API_KEY` et `HUGGINGFACE_REPO_ID` (pour le modèle génératif) sont requis.

         * Si réglé sur `"ollama"` : `OLLAMA_BASE_URL` doit pointer vers votre instance Ollama en cours d'exécution, et que `OLLAMA_GENERATIVE_MODEL_NAME` doit être un modèle que vous avez téléchargé via `ollama pull` et qui est servi par Ollama.

     * **Pour les Embeddings (`DEFAULT_EMBEDDING_PROVIDER`, utilisé par `RetrievalEngine` dans les outils) :**

         * Si réglé sur `"openai"` : `OPENAI_API_KEY` et `OPENAI_EMBEDDING_MODEL_NAME` sont requis.

         * Si réglé sur `"huggingface"` (local Sentence Transformers) : `HUGGINGFACE_EMBEDDING_MODEL_NAME` doit être configuré (aucune clé API spécifique n'est généralement nécessaire pour cette partie).

         * Si réglé sur `"ollama"` : `OLLAMA_BASE_URL` et `OLLAMA_EMBEDDING_MODEL_NAME` (un modèle d'embedding approprié) sont requis et le modèle doit être servi par Ollama.

 * **Checkpointer LangGraph :** Le `MongoDBSaver` est utilisé par défaut par le workflow (`graph_app_v2_1` dans `main_workflow.py`) ; son bon fonctionnement dépend de la configuration correcte de `MONGODB_URI`.

In [1]:
import logging
import sys
from pathlib import Path
import os
import json
import asyncio
import uuid 
import pprint 
from typing import Dict, Any, Optional, List 

project_root = Path()

from dotenv import load_dotenv
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}.")

from config.settings import settings
from config.logging_config import setup_logging
from src.graph.main_workflow import run_makers_v2_1
from src.graph.checkpointer import MongoDBSaver 
from src.vector_store.mongodb_manager import MongoDBManager 

LOG_LEVEL_NOTEBOOK = "INFO" 
setup_logging(level=LOG_LEVEL_NOTEBOOK) 
logger = logging.getLogger("nb_06_e2e_test")

logger.info(f"--- Configuration Active pour le Test de Bout en Bout (depuis settings.py et .env) ---")

generative_llm_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
logger.info(f"Fournisseur LLM génératif principal pour les agents : '{generative_llm_provider}'")
config_llm_ok = False
if generative_llm_provider == "openai":
    if settings.OPENAI_API_KEY and settings.DEFAULT_OPENAI_GENERATIVE_MODEL:
        config_llm_ok = True
        logger.info(f"  OpenAI: Clé API trouvée, Modèle: {settings.DEFAULT_OPENAI_GENERATIVE_MODEL}")
    else:
        logger.error("  ERREUR OpenAI: OPENAI_API_KEY et/ou DEFAULT_OPENAI_GENERATIVE_MODEL manquants.")
elif generative_llm_provider == "huggingface_api":
    if settings.HUGGINGFACE_API_KEY and settings.HUGGINGFACE_REPO_ID:
        config_llm_ok = True
        logger.info(f"  HuggingFace API: Clé API trouvée, Repo ID: {settings.HUGGINGFACE_REPO_ID}")
    else:
        logger.error("  ERREUR HuggingFace API: HUGGINGFACE_API_KEY et/ou HUGGINGFACE_REPO_ID manquants.")
elif generative_llm_provider == "ollama":
    if settings.OLLAMA_BASE_URL and settings.OLLAMA_GENERATIVE_MODEL_NAME:
        config_llm_ok = True
        logger.info(f"  Ollama: URL Base: {settings.OLLAMA_BASE_URL}, Modèle Génératif: {settings.OLLAMA_GENERATIVE_MODEL_NAME}")
        logger.info(f"    (Assurez-vous que le modèle '{settings.OLLAMA_GENERATIVE_MODEL_NAME}' est servi par Ollama via 'ollama pull ...')")
    else:
        logger.error("  ERREUR Ollama: OLLAMA_BASE_URL et/ou OLLAMA_GENERATIVE_MODEL_NAME manquants.")
else:
    logger.error(f"  ERREUR: Fournisseur LLM génératif inconnu : '{generative_llm_provider}'")

if not config_llm_ok:
     logger.warning(f"  AVERTISSEMENT: Configuration LLM pour '{generative_llm_provider}' incomplète. Le workflow risque d'échouer.")

embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Fournisseur d'Embedding (pour RAG via RetrievalEngine) : '{embedding_provider}'")
config_embedding_ok = False
if embedding_provider == "openai":
    if settings.OPENAI_API_KEY and settings.OPENAI_EMBEDDING_MODEL_NAME:
        config_embedding_ok = True
        logger.info(f"  OpenAI Embeddings: Clé API trouvée, Modèle: {settings.OPENAI_EMBEDDING_MODEL_NAME}")
    else:
        logger.error("  ERREUR OpenAI Embeddings: OPENAI_API_KEY et/ou OPENAI_EMBEDDING_MODEL_NAME manquants.")
elif embedding_provider == "huggingface":
    if settings.HUGGINGFACE_EMBEDDING_MODEL_NAME:
        config_embedding_ok = True
        logger.info(f"  HuggingFace Embeddings (local): Modèle: {settings.HUGGINGFACE_EMBEDDING_MODEL_NAME}")
    else:
        logger.error("  ERREUR HuggingFace Embeddings: HUGGINGFACE_EMBEDDING_MODEL_NAME manquant.")
elif embedding_provider == "ollama":
    if settings.OLLAMA_BASE_URL and settings.OLLAMA_EMBEDDING_MODEL_NAME:
        config_embedding_ok = True
        logger.info(f"  Ollama Embeddings: URL Base: {settings.OLLAMA_BASE_URL}, Modèle: {settings.OLLAMA_EMBEDDING_MODEL_NAME}")
        logger.info(f"    (Assurez-vous que le modèle d'embedding '{settings.OLLAMA_EMBEDDING_MODEL_NAME}' est servi par Ollama.)")
    else:
        logger.error("  ERREUR Ollama Embeddings: OLLAMA_BASE_URL et/ou OLLAMA_EMBEDDING_MODEL_NAME manquants.")
else:
    logger.error(f"  ERREUR: Fournisseur d'embedding inconnu : '{embedding_provider}'")

if not config_embedding_ok:
    logger.warning(f"  AVERTISSEMENT: Configuration Embedding pour '{embedding_provider}' incomplète. Le RAG risque d'échouer.")

if not settings.MONGODB_URI or ("<user>" in settings.MONGODB_URI and "<password>" in settings.MONGODB_URI) or "<cluster_url>" in settings.MONGODB_URI:
    logger.error("ERREUR CRITIQUE : MONGODB_URI non trouvé ou semble non configuré (contient des placeholders). Le checkpointer et le RetrievalEngine (RAG) échoueront.")
else:
    logger.info("MongoDB URI configuré.")
    logger.info(f"  Base de données MongoDB: {settings.MONGO_DATABASE_NAME}")
    logger.info(f"  Collection des checkpoints LangGraph: {settings.LANGGRAPH_CHECKPOINTS_COLLECTION}")
    logger.info(f"  Collection des chunks (RAG default): {MongoDBManager.DEFAULT_CHUNK_COLLECTION_NAME}") 

logger.info("--- Fin de la Vérification de Configuration Active ---")

def display_final_synthesis(final_state: Dict[str, Any]):
    print("\n--- Synthèse Finale Produite (ou Erreur) ---")
    if not final_state:
        print("Aucun état final retourné.")
        return
    
    synthesis = final_state.get("synthesis_output")
    error_msg = final_state.get("error_message")

    if synthesis:
        print(synthesis)
    elif error_msg:
        print(f"ERREUR DANS LE WORKFLOW : {error_msg}")
    else:
        print("Aucune synthèse explicite ni message d'erreur trouvé dans les champs dédiés de l'état final.")
        print("Affichage de l'état final complet pour débogage :")
        pprint.pprint(final_state) 
    print("------------------------------------")


ATTENTION: Fichier .env non trouvé à .env.
2025-06-04 00:12:40 - nb_06_e2e_test - INFO - --- Configuration Active pour le Test de Bout en Bout (depuis settings.py et .env) ---
2025-06-04 00:12:40 - nb_06_e2e_test - INFO - Fournisseur LLM génératif principal pour les agents : 'ollama'
2025-06-04 00:12:40 - nb_06_e2e_test - INFO -   Ollama: URL Base: http://localhost:11434, Modèle Génératif: mistral
2025-06-04 00:12:40 - nb_06_e2e_test - INFO -     (Assurez-vous que le modèle 'mistral' est servi par Ollama via 'ollama pull ...')
2025-06-04 00:12:40 - nb_06_e2e_test - INFO - Fournisseur d'Embedding (pour RAG via RetrievalEngine) : 'ollama'
2025-06-04 00:12:40 - nb_06_e2e_test - INFO -   Ollama Embeddings: URL Base: http://localhost:11434, Modèle: nomic-embed-text
2025-06-04 00:12:40 - nb_06_e2e_test - INFO -     (Assurez-vous que le modèle d'embedding 'nomic-embed-text' est servi par Ollama.)
2025-06-04 00:12:40 - nb_06_e2e_test - INFO - MongoDB URI configuré.
2025-06-04 00:12:40 - nb_06_

 ### 1. Définition d'une Requête Utilisateur Complexe et Multi-Facettes



 Nous allons choisir une requête qui nécessite une planification, potentiellement une recherche de nouveaux documents et une analyse de plusieurs aspects avant la synthèse.

In [2]:
# Exemple de requête complexe :
complex_query = (
    "Provide a comprehensive overview of the use of deep reinforcement learning (DRL) for "
    "autonomous drone navigation in complex, cluttered urban environments. "
    "The overview should cover: "
    "1. Key DRL algorithms employed (e.g., PPO, SAC, DDPG variations). "
    "2. Common simulation environments and sim-to-real transfer challenges specific to this domain. "
    "3. How sensor fusion (e.g., vision, LiDAR, IMU) is typically handled in DRL policies for drones. "
    "4. Explicitly search for and include findings from any ArXiv papers published in the last 12-18 months on this topic, "
    "especially those addressing safety or obstacle avoidance. "
    "5. Summarize future research directions."
)

logger.info(f"Requête complexe pour le test de bout en bout : '{complex_query}'")


2025-06-04 00:12:40 - nb_06_e2e_test - INFO - Requête complexe pour le test de bout en bout : 'Provide a comprehensive overview of the use of deep reinforcement learning (DRL) for autonomous drone navigation in complex, cluttered urban environments. The overview should cover: 1. Key DRL algorithms employed (e.g., PPO, SAC, DDPG variations). 2. Common simulation environments and sim-to-real transfer challenges specific to this domain. 3. How sensor fusion (e.g., vision, LiDAR, IMU) is typically handled in DRL policies for drones. 4. Explicitly search for and include findings from any ArXiv papers published in the last 12-18 months on this topic, especially those addressing safety or obstacle avoidance. 5. Summarize future research directions.'


 ### 2. Exécution du Workflow "MAKERS"



 Nous lançons le workflow avec cette requête. Le checkpointer MongoDB sauvegardera les états intermédiaires.

 Nous allons observer les logs (surtout si `LOG_LEVEL_NOTEBOOK` est à `DEBUG` dans la cellule de configuration ou si la fonction `run_makers_v2_1` a sa propre verbosité d'événements activée).

In [3]:
e2e_thread_id = "e2e_test_thread_" + str(uuid.uuid4())

# complex_query est défini dans la cellule précédente.
# LOG_LEVEL_NOTEBOOK est défini dans la première cellule de ce script.
# Les providers LLM et embedding sont lus depuis settings (chargés depuis .env)
print(f"Lancement du workflow de bout en bout pour la requête avec thread_id: {e2e_thread_id}")
print(f"Utilisation du LLM provider configuré: '{settings.DEFAULT_LLM_MODEL_PROVIDER}' et du provider d'embedding: '{settings.DEFAULT_EMBEDDING_PROVIDER}'.")
print(f"Niveau de Log pour ce notebook: '{LOG_LEVEL_NOTEBOOK}'. Surveillez la console pour les logs détaillés du flux d'agents et des appels d'outils...")
print("Le traitement de la requête complexe peut prendre plusieurs minutes...")

# --- Gestion asyncio pour Jupyter ---
# Nécessaire si asyncio.run() est appelé dans un environnement avec une boucle d'événements déjà active (comme Jupyter)
import nest_asyncio
nest_asyncio.apply()
# --- Fin Gestion asyncio ---

final_state_e2e = None

# Vérification principale: MONGODB_URI est essentiel pour le checkpointer et souvent pour les outils RAG.
# Les configurations LLM/Embedding ont été vérifiées (et des logs émis) dans la première cellule de ce notebook.
# Les erreurs d'instanciation dues à des configurations manquantes pour ces services seront attrapées par le try/except.
if not settings.MONGODB_URI or ("<user>" in settings.MONGODB_URI and "<password>" in settings.MONGODB_URI) or "<cluster_url>" in settings.MONGODB_URI:
    print("\nERREUR CRITIQUE: MONGODB_URI n'est pas configuré correctement dans le fichier .env (il manque ou contient des placeholders comme <user>).")
    print("L'exécution du workflow est annulée car le checkpointer MongoDB et potentiellement les outils RAG sont requis.")
    logger.error("MONGODB_URI non configuré ou contient des placeholders. Workflow de bout en bout non exécuté.")
else:
    try:
        # La fonction run_makers_v2_1 est importée depuis src.graph.main_workflow
        # Elle utilisera les providers LLM et embedding configurés via settings.py (et .env).
        # Les erreurs de configuration (clés API, URLs, modèles non trouvés) seront levées par 
        # les modules sous-jacents (llm_factory, RetrievalEngine) et attrapées ici.
        logger.info(f"Appel de run_makers_v2_1 avec la requête: \"{complex_query[:100]}...\" et thread_id: {e2e_thread_id}")
        final_state_e2e = asyncio.run(run_makers_v2_1(complex_query, thread_id=e2e_thread_id))
        
    except ValueError as ve: # Pour les erreurs de configuration de get_llm ou RetrievalEngine
        logger.error(f"Erreur de configuration (ValueError) lors de l'exécution du workflow de bout en bout: {ve}", exc_info=True)
        print(f"\nERREUR DE CONFIGURATION PENDANT L'EXÉCUTION DU WORKFLOW : {ve}")
        print("Veuillez vérifier les configurations pour DEFAULT_LLM_MODEL_PROVIDER, DEFAULT_EMBEDDING_PROVIDER, ")
        print("et leurs dépendances respectives (clés API, URLs de base, noms de modèles exacts) dans votre fichier .env et settings.py.")
        print(f"Provider LLM actuel: {settings.DEFAULT_LLM_MODEL_PROVIDER}, Provider Embedding actuel: {settings.DEFAULT_EMBEDDING_PROVIDER}")
    except RuntimeError as re: # Pour les erreurs spécifiques à asyncio si nest_asyncio ne suffit pas
        if "cannot be called from a running event loop" in str(re):
            logger.error(f"Erreur RuntimeError avec asyncio.run(): {re}. 'nest_asyncio.apply()' n'a peut-être pas été appelé ou n'a pas fonctionné.", exc_info=True)
            print(f"\nERREUR ASYNCIO : {re}. Assurez-vous que 'nest_asyncio' est installé et que 'nest_asyncio.apply()' est appelé avant 'asyncio.run()'.")
        else:
            logger.error(f"Erreur RuntimeError inattendue lors de l'exécution du workflow: {re}", exc_info=True)
            print(f"\nERREUR RUNTIME INATTENDUE PENDANT L'EXÉCUTION DU WORKFLOW : {re}")
    except Exception as e: # Pour les autres erreurs d'exécution inattendues
        logger.error(f"Erreur inattendue lors de l'exécution du workflow de bout en bout: {e}", exc_info=True)
        print(f"\nERREUR INATTENDUE PENDANT L'EXÉCUTION DU WORKFLOW : {e}")

# Afficher la synthèse finale (ou l'erreur)
# La fonction display_final_synthesis est définie dans la première cellule de code de ce notebook.
if final_state_e2e:
    display_final_synthesis(final_state_e2e) 
else:
    print("\nL'exécution du workflow n'a pas retourné d'état final ou a échoué avant de pouvoir retourner un état.")
    if not settings.MONGODB_URI or ("<user>" in settings.MONGODB_URI and "<password>" in settings.MONGODB_URI) or "<cluster_url>" in settings.MONGODB_URI:
        print("Rappel : MONGODB_URI n'était pas (ou mal) configuré, ce qui a pu empêcher l'exécution.")


Lancement du workflow de bout en bout pour la requête avec thread_id: e2e_test_thread_c27e0304-7a0c-4fd0-a85e-491450b397f3
Utilisation du LLM provider configuré: 'ollama' et du provider d'embedding: 'ollama'.
Niveau de Log pour ce notebook: 'INFO'. Surveillez la console pour les logs détaillés du flux d'agents et des appels d'outils...
Le traitement de la requête complexe peut prendre plusieurs minutes...
2025-06-04 00:12:40 - nb_06_e2e_test - INFO - Appel de run_makers_v2_1 avec la requête: "Provide a comprehensive overview of the use of deep reinforcement learning (DRL) for autonomous dron..." et thread_id: e2e_test_thread_c27e0304-7a0c-4fd0-a85e-491450b397f3
2025-06-04 00:12:41 - src.graph.checkpointer - INFO - MongoDBSaver initialized for database 'makers_db', collection 'langgraph_checkpoints'.
2025-06-04 00:12:42 - src.graph.checkpointer - WARNING - parent_config provided, but its 'configurable' key was missing/not a dict, AND 'id' key was not found/valid in parent_config itself.

 ### 3. Analyse Qualitative des Sorties Intermédiaires



 Si l'exécution précédente a réussi et retourné `final_state_e2e`, nous pouvons examiner certains des champs clés de cet état pour comprendre le comportement du système.

In [4]:
if final_state_e2e and not final_state_e2e.get("error_message"):
    print("\n--- Analyse des Sorties Intermédiaires Clés ---")

    # 1. Plan de Recherche
    research_plan = final_state_e2e.get("research_plan")
    if research_plan:
        print("\n### Plan de Recherche Généré par ResearchPlannerAgent ###")
        # Pour un affichage potentiellement long, on peut tronquer ou utiliser IPython.display.Markdown si c'est du Markdown
        if isinstance(research_plan, str) and ("\n##" in research_plan or "\n*" in research_plan):
            try:
                from IPython.display import display, Markdown
                display(Markdown(research_plan))
            except ImportError:
                print(research_plan)
        else:
            print(research_plan)
    else:
        print("\nAucun plan de recherche explicite trouvé dans l'état final.")

    # 2. Analyse des Messages (pour les résultats d'outils et les pensées des agents)
    print("\n### Analyse des Messages Clés de l'Exécution ###")
    messages = final_state_e2e.get("messages", [])
    
    if not messages:
        print("Aucun message dans l'état final.")
    else:
        # Afficher les quelques derniers messages pour voir le contexte final
        # La fonction pretty_print_final_state de la première cellule de 04_... était plus détaillée ici.
        # Pour cette cellule, on se concentre sur les ToolMessages.
        print(f"Nombre total de messages: {len(messages)}. Affichage des ToolMessages et des derniers AIMessages:")

        for i, msg in enumerate(messages):
            msg_type_str = getattr(msg, 'type', 'UNKNOWN').upper()
            msg_name_str = getattr(msg, 'name', None)
            display_name = f"{msg_type_str} ({msg_name_str})" if msg_name_str else msg_type_str

            if msg_type_str == "TOOL":
                tool_call_id = getattr(msg, 'tool_call_id', 'N/A')
                print(f"\n  Message #{i+1}: [{display_name}] - Tool Call ID: {tool_call_id}")
                tool_content_str = str(getattr(msg, 'content', 'N/A'))
                try:
                    # Tenter de parser si c'est une chaîne JSON (pour les outils structurés)
                    if tool_content_str.strip().startswith(("{", "[")):
                        tool_content_parsed = json.loads(tool_content_str)
                        print("    Contenu (parsé en JSON):")
                        print(json.dumps(tool_content_parsed, indent=2, ensure_ascii=False))
                        
                        # Heuristique pour identifier le type d'outil basé sur le contenu
                        if isinstance(tool_content_parsed, list) and tool_content_parsed:
                            if isinstance(tool_content_parsed[0], dict):
                                if "pdf_url" in tool_content_parsed[0]:
                                    print(f"    (Semble être un résultat de ArXiv Search - {len(tool_content_parsed)} items)")
                                elif "text_chunk" in tool_content_parsed[0]:
                                    print(f"    (Semble être un résultat de KB Retrieval - {len(tool_content_parsed)} chunks)")
                    else: # Si ce n'est pas du JSON évident, afficher comme chaîne
                        print(f"    Contenu (chaîne): {tool_content_str[:500]}{'...' if len(tool_content_str) > 500 else ''}")
                except json.JSONDecodeError:
                    print(f"    Contenu (chaîne non-JSON): {tool_content_str[:500]}{'...' if len(tool_content_str) > 500 else ''}")
                except Exception as e_parse:
                    print(f"    Impossible d'analyser/afficher le contenu de ToolMessage : {e_parse}")
            
            # Optionnel: Afficher les derniers messages d'IA non-tool-calling (pour voir les "pensées" finales des agents)
            elif msg_type_str == "AI" and not getattr(msg, 'tool_calls', None) and i >= len(messages) - 3 : # Derniers 3 messages
                 print(f"\n  Message #{i+1}: [{display_name}] (Pensée/Réponse finale de l'agent)")
                 print(f"    Contenu: {str(getattr(msg, 'content', 'N/A'))[:500]}{'...' if len(str(getattr(msg, 'content', 'N/A'))) > 500 else ''}")


    # 3. Résumé de l'Analyse de Documents (si produit par DocumentAnalysisAgent sans être un appel d'outil direct)
    doc_analysis_summary = final_state_e2e.get("document_analysis_summary")
    if doc_analysis_summary:
        print("\n### Résumé de l'Analyse de Documents (champ 'document_analysis_summary') ###")
        # Ce champ peut contenir du Markdown si le document_deep_dive_analysis_tool a été utilisé
        if isinstance(doc_analysis_summary, str) and ("\n##" in doc_analysis_summary or "\n*" in doc_analysis_summary):
             try:
                from IPython.display import display, Markdown
                display(Markdown(doc_analysis_summary))
             except ImportError:
                print(doc_analysis_summary)
        else:
            print(doc_analysis_summary)
    else:
        print("\nAucun résumé d'analyse de document explicite trouvé dans le champ 'document_analysis_summary' de l'état final.")
    
    print("\n--- Fin de l'Analyse des Sorties Intermédiaires ---")

elif final_state_e2e and final_state_e2e.get("error_message"):
    # Ce message est déjà géré par display_final_synthesis dans la cellule précédente
    print(f"\nL'exécution du workflow a produit une erreur (voir message dans la sortie de la cellule précédente). Analyse des sorties intermédiaires impossible.")
else:
    print("\nÉtat final ('final_state_e2e') non disponible ou vide. Impossible d'analyser les sorties intermédiaires.")
    print("Veuillez exécuter la cellule précédente (exécution du workflow) avec succès.")



--- Analyse des Sorties Intermédiaires Clés ---

Aucun plan de recherche explicite trouvé dans l'état final.

### Analyse des Messages Clés de l'Exécution ###
Aucun message dans l'état final.

Aucun résumé d'analyse de document explicite trouvé dans le champ 'document_analysis_summary' de l'état final.

--- Fin de l'Analyse des Sorties Intermédiaires ---


 ### 4. Inspection des Checkpoints dans MongoDB



 Si le `MongoDBSaver` est actif (ce qui est le cas par défaut dans notre `main_workflow.py`), nous pouvons interroger MongoDB pour voir les états sauvegardés pour le `thread_id` de cette exécution.

In [5]:
# --- AJOUT DE L'IMPORT MANQUANT ---
from pymongo.errors import ConnectionFailure 

async def inspect_checkpoints(thread_id: str):
    logger.info(f"\n--- Inspection des Checkpoints pour Thread ID: {thread_id} ---")
    if not settings.MONGODB_URI: # MONGODB_URI est vérifié aussi dans la 1ère cellule, mais redondance ici est ok.
        logger.error("MONGODB_URI non configuré. Impossible d'inspecter les checkpoints.")
        print("ERREUR: MONGODB_URI non configuré. Inspection des checkpoints annulée.")
        return

    checkpointer = None 
    try:
        # MongoDBSaver est importé dans la première cellule de ce notebook
        checkpointer = MongoDBSaver(
            collection_name=settings.LANGGRAPH_CHECKPOINTS_COLLECTION 
        )
        
        logger.info(f"Récupération des checkpoints pour thread_id='{thread_id}' depuis la collection '{settings.LANGGRAPH_CHECKPOINTS_COLLECTION}'...")
        
        config_for_list = {"configurable": {"thread_id": thread_id}}
        checkpoints_history = []
        async for checkpoint_tuple in checkpointer.alist(config=config_for_list):
            checkpoints_history.append(checkpoint_tuple)
        
        if not checkpoints_history:
            print(f"Aucun checkpoint trouvé pour le thread_id: {thread_id}")
            return

        print(f"\nTrouvé {len(checkpoints_history)} checkpoints pour le thread_id: {thread_id} (du plus récent au plus ancien):")
        
        for i, cp_tuple in enumerate(checkpoints_history[:3]): 
            checkpoint_id_ts = cp_tuple.checkpoint.get('id', 'N/A') 
            
            print(f"\nCheckpoint #{i+1} (ts/id: {checkpoint_id_ts}):")
            print(f"  Config du checkpoint: {cp_tuple.config}")
            print(f"  Metadata: {cp_tuple.metadata}")
            
            parent_ts_info = "None"
            if cp_tuple.parent_config and cp_tuple.parent_config.get("configurable"):
                parent_ts_info = cp_tuple.parent_config["configurable"].get('thread_ts', 'N/A')
            print(f"  Parent ts (depuis parent_config): {parent_ts_info}")
            
            messages_in_checkpoint = cp_tuple.checkpoint.get("channel_values", {}).get("messages", [])
            if messages_in_checkpoint:
                last_msg_in_cp = messages_in_checkpoint[-1]
                msg_type = getattr(last_msg_in_cp, 'type', 'UNKNOWN').upper()
                msg_name = getattr(last_msg_in_cp, 'name', '') 
                msg_content = str(getattr(last_msg_in_cp, 'content', ''))
                print(f"  Dernier message dans ce checkpoint: [{msg_type}{' ('+msg_name+')' if msg_name else ''}]: {msg_content[:100]}...")
            else:
                print("  Aucun message trouvé dans channel_values pour ce checkpoint.")
        
        if len(checkpoints_history) > 3:
            print(f"\n... et {len(checkpoints_history) - 3} checkpoint(s) plus ancien(s) non affiché(s) en détail.")

    except ConnectionFailure as cf: 
        logger.error(f"Erreur de connexion MongoDB lors de l'inspection des checkpoints: {cf}", exc_info=True)
        print(f"ERREUR DE CONNEXION MONGODB: {cf}")
    except Exception as e:
        logger.error(f"Erreur lors de l'inspection des checkpoints: {e}", exc_info=True)
        print(f"Erreur inattendue lors de l'inspection des checkpoints: {e}")
    finally:
        if checkpointer and hasattr(checkpointer, 'aclose'): 
            await checkpointer.aclose()
            logger.info("Connexion du checkpointer MongoDB fermée.")

if 'e2e_thread_id' in locals() and e2e_thread_id:
    print(f"\nTentative d'inspection des checkpoints pour le thread_id: {e2e_thread_id}")
    # nest_asyncio.apply() a été appelé dans la cellule d'exécution du workflow ci-dessus.
    # Si cette cellule est exécutée indépendamment après un redémarrage du noyau, 
    # il faudrait décommenter les lignes nest_asyncio ci-dessous.
    # import nest_asyncio 
    # nest_asyncio.apply() 
    
    asyncio.run(inspect_checkpoints(e2e_thread_id))
else:
    logger.warning("'e2e_thread_id' non défini. L'exécution précédente du workflow a peut-être échoué ou cette cellule est exécutée hors séquence.")
    print("\nVariable 'e2e_thread_id' non trouvée. Exécutez d'abord la cellule d'exécution du workflow principal pour définir un thread_id.")



Tentative d'inspection des checkpoints pour le thread_id: e2e_test_thread_c27e0304-7a0c-4fd0-a85e-491450b397f3
2025-06-04 00:13:33 - nb_06_e2e_test - INFO - 
--- Inspection des Checkpoints pour Thread ID: e2e_test_thread_c27e0304-7a0c-4fd0-a85e-491450b397f3 ---
2025-06-04 00:13:33 - src.graph.checkpointer - INFO - MongoDBSaver initialized for database 'makers_db', collection 'langgraph_checkpoints'.
2025-06-04 00:13:33 - nb_06_e2e_test - INFO - Récupération des checkpoints pour thread_id='e2e_test_thread_c27e0304-7a0c-4fd0-a85e-491450b397f3' depuis la collection 'langgraph_checkpoints'...

Trouvé 6 checkpoints pour le thread_id: e2e_test_thread_c27e0304-7a0c-4fd0-a85e-491450b397f3 (du plus récent au plus ancien):

Checkpoint #1 (ts/id: 1f040c7f-ccd1-6f58-8004-42d651f07bd9):
  Config du checkpoint: {'configurable': {'thread_id': 'e2e_test_thread_c27e0304-7a0c-4fd0-a85e-491450b397f3', 'thread_ts': '1f040c7f-ccd1-6f58-8004-42d651f07bd9'}}
  Metadata: {'source': 'loop', 'writes': {'synthe

 ### 5. Discussion et Analyse Qualitative de la Synthèse Finale



 Revenons à la synthèse finale produite à l'étape 2 (stockée dans `final_state_e2e['synthesis_output']`).

 * La synthèse répond-elle de manière complète et précise à la requête complexe initiale ?

 * Les différents aspects de la requête (algorithmes DRL, environnements de simulation, défis sim-to-real, fusion de capteurs, résultats récents d'ArXiv, directions futures) sont-ils couverts ?

 * L'information est-elle bien structurée et cohérente ?

 * Y a-t-il des signes d'hallucination ou des informations manquantes cruciales (en supposant que le corpus contient les informations nécessaires) ?



 Cette analyse qualitative est subjective mais essentielle pour comprendre les forces et faiblesses actuelles du système. Elle peut guider les améliorations des prompts des agents, de la logique de routage, ou des stratégies RAG.

 ## Conclusion de ce Test de Bout en Bout



 Ce notebook a permis d'exécuter le "MAKERS" sur une requête complexe, d'examiner certaines sorties intermédiaires et la synthèse finale, et de voir comment les checkpoints sont gérés.



 Ce type de test approfondi est utile pour :

 - Identifier les goulots d'étranglement ou les points faibles dans le flux des agents.

 - Évaluer qualitativement la performance globale.

 - Déboguer des comportements inattendus.

 - Générer des exemples concrets pour l'évaluation quantitative (par exemple, des paires `(requête, contexte, synthèse)` pour `SynthesisEvaluator`).